# Notes

**What we need for the implementation**
- Relational Database Schema
    - R set of relations
    - K function that maps each relation to its set of primary key attributes
    - F function that maps each relation to its set of attributes, 
        where each attribute is given by its column name together with its datatype
- Each relation has:
    - Atribute set (F)
    - pk set (K)
    - Foreign key set (subset of F)
- Functions:
    - docTable: returns the textual description of a relation
    - docAttr: returns the textual description of an attribute a in relation r
- Ontology Representation:
    - C set of class identifiers (concepts)
    - P set of properties
        - Pobj subset of object properties
        - Pdata subset of data properties
    - A set of axioms 
        - class hierarchies, 
        - domain and range assertions, 
        - property characteristics
    - M set of annotation assertions
        - labels
        - comments
        - provenance info
- External Ontology Repo
    - DINGO
- Lexical view converter 
    - of any ontology
    - **CHECK**
- RAG pipeline 
    - Embed lexical views of 
        - external ontologies
        - Relational schema 
        - core ontology
    - Index in FAISS 
    - At each relation (...)

# Data Sctructures

## Prompt

In [ ]:
prompt = f"""
    Generate ontology elements with provenance annotations for database table {data[table_name]} based on:

    [CONTEXT]
    - Database Schema of the database {data[schema_context]}
    - Take semantics from the Relevant Documents {data[documents]}
    - Take semantics from the Existing Ontology Knowledge {data[existing_ontology]}

    [INSTRUCTIONS]
    1. Include these elements:
        Classes (subclass of Thing)
        Data properties with domain/range
        Object properties with domain/range
        Use only one rdfs:domain and one rdfs:range per property. If multiple options exist, select the most general or create a shared superclass.
    3. Do not create a property named "is". Use rdf:type for instance membership, rdfs:subClassOf for class hierarchies, and owl:sameAs for instance equality.
    4. Use this format example:

    Class: {data[table_name]}
    Annotations:
    prov:wasDerivedFrom
    <http://example.org/provenance/{data[table_name]}>

    DataProperty:
    has_column_name
    domain {data[table_name]}
    range string
    Annotations:
    prov:wasDerivedFrom
    <http://example.org/provenance/{data[table_name]}/column_name>

    ObjectProperty:
    relates_to_table domain {data[table_name]}
    range RelatedTable
    Annotations:
    prov:wasDerivedFrom
    <http://example.org/provenance/{data[table_name]}/fk_column>

    Only output Manchester Syntax and nothing else. [OUTPUT]
    {core_ontology}
    {table_schema}
    {external_ontology}
"""

## Database Schema

In [ ]:
import re
from pprint import pprint

def parse_mysql_ddl_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        ddl = f.read()

    # Remove comments and MySQL directives
    ddl = re.sub(r'/\*.*?\*/', '', ddl, flags=re.DOTALL)
    ddl = re.sub(r'--.*?$', '', ddl, flags=re.MULTILINE)
    ddl = re.sub(r'/\!.*?\*/;', '', ddl, flags=re.DOTALL)
    ddl = re.sub(r'/\!.*?\*/', '', ddl, flags=re.DOTALL)

    # Find all CREATE TABLE statements (handles backticks and multiline)
    table_regex = re.compile(
        r'CREATE TABLE\s+`?(\w+)`?\s*\((.*?)\)\s*ENGINE=.*?;',
        re.DOTALL | re.IGNORECASE
    )
    tables = table_regex.findall(ddl)
    result = {}

    for table_name, table_body in tables:
        # Raw DDL
        raw_ddl = f"CREATE TABLE `{table_name}` ({table_body});"

        # Split lines, remove empty and trailing commas
        lines = [line.strip().rstrip(',') for line in table_body.splitlines() if line.strip()]
        columns = []
        primary_keys = []
        foreign_keys = []

        for line in lines:
            # Column definition (starts with backtick or word, not constraint)
            if re.match(r'^`?\w+`?\s', line) and not line.upper().startswith(('PRIMARY KEY', 'FOREIGN KEY', 'CONSTRAINT', 'UNIQUE', 'KEY')):
                col_name = re.match(r'^`?(\w+)`?', line).group(1)
                columns.append(col_name)
            # Primary key
            elif line.upper().startswith('PRIMARY KEY'):
                pk_match = re.search(r'\((.*?)\)', line)
                if pk_match:
                    pk_cols = [col.strip(' `') for col in pk_match.group(1).split(',')]
                    primary_keys.extend(pk_cols)
            # Foreign key
            elif line.upper().startswith('CONSTRAINT') and 'FOREIGN KEY' in line.upper():
                fk_match = re.search(r'FOREIGN KEY\s*\((.*?)\)\s*REFERENCES\s*`?(\w+)`?\s*\((.*?)\)', line, re.IGNORECASE)
                if fk_match:
                    fk_cols = [col.strip(' `') for col in fk_match.group(1).split(',')]
                    ref_table = fk_match.group(2)
                    ref_cols = [col.strip(' `') for col in fk_match.group(3).split(',')]
                    foreign_keys.append({
                        'columns': fk_cols,
                        'ref_table': ref_table,
                        'ref_columns': ref_cols
                    })

        result[table_name] = {
            'raw_ddl': raw_ddl,
            'columns': columns,
            'primary_keys': primary_keys,
            'foreign_keys': foreign_keys
        }

    return result

In [12]:
schema = parse_mysql_ddl_file('../RDB_schema/Usable_schema.sql')
pprint(schema)

{'granter_activity': {'columns': ['id',
                                  'created_at',
                                  'updated_at',
                                  'type',
                                  'title',
                                  'description',
                                  'application_id',
                                  'company_id',
                                  'file_id',
                                  'profile_id',
                                  'created_by_expert',
                                  'activity_date',
                                  'data',
                                  'data_id',
                                  'opportunity_id',
                                  'data_type'],
                      'foreign_keys': [{'columns': ['application_id'],
                                        'ref_columns': ['id'],
                                        'ref_table': 'granter_application'},
                                 

## Descriptions of the Database tables

In [13]:
from langchain_ollama import OllamaLLM
llm = OllamaLLM(model="llama3.2:3b")

c:\Users\tiago\anaconda3\envs\Graph_implementation\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
